## Making the red SPIRE catalogue

![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>)


### The method we used is decsribed in Asboth et al. 2016. We recommend the user to read this paper before using the data products.
### Our method creates a 6 arcsec 500um map by using bicubic interpolation. Than a D-map is created with D  = 0.92M500 − 0.392M250, with M500 the 500um map and M250 the 250um map.
### The D-map is created in the same way as Asboth et al. 2016, and we select all sources with a 3sigma detection in the D-map. We extract the flux density of the sources by using the same method as used for the standart blind catalogue. 

### The data product created is: "name_field"_D_MF_cat.fits, and contains all the 3sigma detecion in the D-map, therfore this catalogue is very complete, but also has a very low reliability.

### The RED SPIRE sources are validated with the 'red_validation.ipynb' notebook, which contains cuts to find reliable 500um riser candidates. 

In [5]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
import os

import matplotlib.pyplot as plt
from astropy.wcs import WCS
from matplotlib.patches import Ellipse
from astropy.coordinates import SkyCoord
from astropy import units as u
%matplotlib inline  
from reproject import reproject_interp
from reproject import reproject_exact
import sys
sys.path.append('../') # location of func_make_cat.py
import func_make_cat as fc




In [10]:
all_names =  ['GAMA-09_SPIRE','GAMA-12_SPIRE','GAMA-15_SPIRE','HATLAS-NGP_SPIRE','HATLAS-SGP_SPIRE','SSDF_SPIRE',\
              'AKARI-SEP_SPIRE','Bootes_SPIRE','CDFS-SWIRE_SPIRE','COSMOS_SPIRE','EGS_SPIRE',\
              'ELAIS-N1_SPIRE','ELAIS-N2_SPIRE','ELAIS-S1_SPIRE','HDF-N_SPIRE','Lockman-SWIRE_SPIRE','SA13_SPIRE',\
              'SPIRE-NEP_SPIRE','xFLS_SPIRE','XMM-13hr_SPIRE','XMM-LSS_SPIRE','AKARI-NEP_SPIRE']
all_names = ['EGS_SPIRE']

band = ['250','350','500']
        

In [12]:
for j in range(np.size(all_names)):  # loops over the HELP fields
    loc = '../../dmu19/dmu19_HELP-SPIRE-maps/data/' # location of help fields

    print(all_names[j]+band[0]+'_v1.0.fits')
    name = all_names[j]+band[0]+'_v1.0.fits'
    hdulist250 = fits.open(loc+name)
    
    name = all_names[j]+band[1]+'_v1.0.fits'
    hdulist350 = fits.open(loc+name)

    name = all_names[j]+band[2]+'_v1.0.fits'
    hdulist500 = fits.open(loc+name)

    # https://reproject.readthedocs.io/en/stable/ for info about the reproject function
    # order 0 pics the nearest pixel from the projected header
    # we use the order 0 reproject map to find the masked pixels (NaN's), the order 2 interplolation requiers
    # the NaN's to ber removed. 
    new_image0, footprint0 = reproject_interp(hdulist500["MFILT"], hdulist250['IMAGE'].header, order = 0) 

    # NaN pixels in the orignal 500um 
    # These locations are needed as the Bicubic interpolation needs NaN's to be set to 0
    bad = np.isnan(hdulist500["MFILT"].data)
    hdulist500["MFILT"].data[bad] = 0 

    # Bicubic interplolation of the 500um map to the 250um map fits header and pixel size
    # Note that Asboth et al. used 500um maps which where made with 6 arcsec resolution, where we use
    # a bicubic interplotations scheme to got from the 500um resolution to 6 arcsec
    new_image2, footprint2 = reproject_interp(hdulist500["MFILT"], hdulist250['IMAGE'].header, order = 2) 
    bad = np.isnan(new_image0)
    new_image2[bad] = np.nan

    # uses a matched filter on the 500um and 250um map and calculated D
    new_D = fc.do_filtering(hdulist250,hdulist500,new_image2)
    
    new_image2 -= np.nanmean(new_image2) # make sure the new image is mean substracted
    Dhdu = fits.ImageHDU(header=hdulist250['IMAGE'].header,data=new_image2)
    Dhdu.writeto('data/'+all_names[j]+'_map_500_6ac.fits') 

    Dhdu = fits.ImageHDU(header=hdulist250['IMAGE'].header,data=new_D)
    Dhdu.writeto('data/'+all_names[j]+'_D_map.fits') 
    Dhdu = fits.open('data/'+all_names[j]+'_D_map.fits')
    # select 3 sigma peaks
    dmin = 3*np.nanstd(Dhdu[1].data)
    dp, rap, decp, xp, yp = fc.find_peak_red(Dhdu,dmin)
    

    # for the AKARI-NEP a region file has to be used due to the very noisy edges 
    if all_names[j] == 'AKARI-NEP_SPIRE':
        import pyregion
        x_all = np.arange(0,np.shape(Dhdu[1].data)[0])
        y_all = np.arange(0,np.shape(Dhdu[1].data)[1])
        x_mat = np.tile(x_all,np.size(y_all))
        y_mat = np.repeat(y_all,np.size(x_all))
    
        region_name = "AKARI-NEP.reg"
        r = pyregion.open(region_name)
        r250 = r.get_filter(hdulist250[1].header)
        
        mask = r250.inside(y_mat,x_mat)
        Dhdu[1].data[x_mat[~mask],y_mat[~mask]] = np.nan
        Dhdu.writeto('data/'+all_names[j]+'_D_map3.fits') 

        dmin = 3*np.nanstd(Dhdu[1].data)
        dp, rap, decp, xp, yp = np.array(fc.find_peak_red(Dhdu,dmin))        



    w1 = WCS(hdulist250["NEBFILT"].header) 
    w2 = WCS(hdulist350["NEBFILT"].header)
    w3 = WCS(hdulist500["MFILT"].header)    
    
    x_250, y_250 = np.round(w1.wcs_world2pix(rap,  decp, 0)).astype(int)
    x_350, y_350 = np.round(w2.wcs_world2pix(rap,  decp, 0)).astype(int)
    x_500, y_500 = np.round(w3.wcs_world2pix(rap,  decp, 0)).astype(int)
    # pixel flux density at peak location
    S250 = hdulist250["NEBFILT"].data[y_250,x_250]
    E250 = hdulist250["ERROR"].data[y_250,x_250]
    S350 = hdulist350["NEBFILT"].data[y_350,x_350]
    E350 = hdulist350["ERROR"].data[y_350,x_350]
    S500 = hdulist500["NEBFILT"].data[y_500,x_500]
    E500 = hdulist500["ERROR"].data[y_500,x_500]

    hdu = fits.BinTableHDU.from_columns(\
         [fits.Column(name='RA', array=rap, format ='F'),
         fits.Column(name='Dec', array=decp, format='F'),
         fits.Column(name='F_RED_pix_SPIRE_250', array=S250, format ='F'),
         fits.Column(name='FErr_RED_pix_SPIRE_250', array=E250, format ='F'),          
         fits.Column(name='F_RED_pix_SPIRE_350', array=S350, format ='F'),
         fits.Column(name='FErr_RED_pix_SPIRE_350', array=E350, format ='F'),
         fits.Column(name='F_RED_pix_SPIRE_500', array=S500, format ='F'),
         fits.Column(name='FErr_RED_pix_SPIRE_500', array=E500, format ='F')         
         ])
    hdu.writeto('data/'+all_names[j]+'_D_cat.fits')
    
    
    hdulist250.close()
    hdulist350.close()
    hdulist500.close()


EGS_SPIRE250_v1.0.fits


In [16]:
# fids matched filter flux densities on sub-pixel scale
for j in range(np.size(all_names)):   
    print(loc+all_names[j]+band[0]+'_v1.0.fits')

    name = all_names[j]+band[0]+'_v1.0.fits'
    hdulist1 = fits.open(loc+name)
    name = all_names[j]+band[1]+'_v1.0.fits'
    hdulist2 = fits.open(loc+name)
    name = all_names[j]+band[2]+'_v1.0.fits'
    hdulist3 = fits.open(loc+name)

    ff = fits.open('data/'+all_names[j]+'_D_cat.fits')
    ra,dec = (ff[1].data)['RA'],(ff[1].data)['Dec']

    r_1, ra_1, dec_1, S250_1, E250_1, S350_1, E350_1, S500_1, E500_1 = \
                fc.corr_psf_max_MF(hdulist1,hdulist2,hdulist3,ra, dec)
    hdu = fits.BinTableHDU.from_columns(\
         [fits.Column(name='RA', array=ra_1, format ='F',unit='degree'),
         fits.Column(name='Dec', array=dec_1, format='F',unit='degree'),
         fits.Column(name='F_RED_MF_SPIRE_250', array=1000*S250_1, format ='F',unit='mJy'),
         fits.Column(name='FErr_RED_MF_SPIRE_250', array=1000*E250_1, format ='F',unit='mJy'),
         fits.Column(name='F_RED_MF_SPIRE_350', array=1000*S350_1, format ='F',unit='mJy'),
         fits.Column(name='FErr_RED_MF_SPIRE_350', array=1000*E350_1, format ='F',unit='mJy'),
         fits.Column(name='F_RED_MF_SPIRE_500', array=1000*S500_1, format ='F',unit='mJy'),
         fits.Column(name='FErr_RED_MF_SPIRE_500', array=1000*E500_1, format ='F',unit='mJy'),
         fits.Column(name='r', array=r_1, format ='F'),
         fits.Column(name='RA_pix', array=(ff[1].data)['RA'], format ='F'), 
         fits.Column(name='Dec_pix', array=(ff[1].data)['Dec'], format ='F'), 
         fits.Column(name='F_RED_pix_SPIRE_250', array=1000*(ff[1].data)['F_RED_pix_SPIRE_250'], format ='F',unit='mJy'), 
         fits.Column(name='FErr_RED_pix_SPIRE_250', array=1000*(ff[1].data)['FErr_RED_pix_SPIRE_250'], format ='F',unit='mJy'), 
         fits.Column(name='F_RED_pix_SPIRE_350', array=1000*(ff[1].data)['F_RED_pix_SPIRE_350'], format ='F',unit='mJy'), 
         fits.Column(name='FErr_RED_pix_SPIRE_350', array=1000*(ff[1].data)['FErr_RED_pix_SPIRE_350'], format ='F',unit='mJy'), 
         fits.Column(name='F_RED_pix_SPIRE_500', array=1000*(ff[1].data)['F_RED_pix_SPIRE_500'], format ='F',unit='mJy'), 
         fits.Column(name='FErr_RED_pix_SPIRE_500', array=1000*(ff[1].data)['FErr_RED_pix_SPIRE_500'], format ='F',unit='mJy') 
         ])
    hdu.writeto('data/'+all_names[j]+'_D_MF_cat.fits') 

../../dmu19/dmu19_HELP-SPIRE-maps/data/EGS_SPIRE250_v1.0.fits
(0.0, '%')
(0.10649627263045794, '%')
(0.21299254526091588, '%')
(0.3194888178913738, '%')
(0.42598509052183176, '%')
(0.5324813631522897, '%')
(0.6389776357827476, '%')
(0.7454739084132056, '%')
(0.8519701810436635, '%')
(0.9584664536741214, '%')
(1.0649627263045793, '%')
(1.1714589989350372, '%')
(1.2779552715654952, '%')
(1.384451544195953, '%')
(1.4909478168264112, '%')
(1.5974440894568689, '%')
(1.703940362087327, '%')
(1.810436634717785, '%')
(1.9169329073482428, '%')
(2.023429179978701, '%')
(2.1299254526091587, '%')
(2.2364217252396164, '%')
(2.3429179978700745, '%')
(2.4494142705005326, '%')
(2.5559105431309903, '%')
(2.6624068157614484, '%')
(2.768903088391906, '%')
(2.8753993610223643, '%')
(2.9818956336528224, '%')
(3.08839190628328, '%')
(3.1948881789137378, '%')
(3.301384451544196, '%')
(3.407880724174654, '%')
(3.5143769968051117, '%')
(3.62087326943557, '%')
(3.727369542066028, '%')
(3.8338658146964857, '%')


(34.07880724174654, '%')
(34.185303514377, '%')
(34.291799787007456, '%')
(34.398296059637914, '%')
(34.504792332268366, '%')
(34.611288604898824, '%')
(34.71778487752928, '%')
(34.82428115015975, '%')
(34.93077742279021, '%')
(35.03727369542066, '%')
(35.14376996805112, '%')
(35.250266240681576, '%')
(35.356762513312034, '%')
(35.46325878594249, '%')
(35.56975505857295, '%')
(35.67625133120341, '%')
(35.78274760383386, '%')
(35.88924387646432, '%')
(35.99574014909478, '%')
(36.102236421725244, '%')
(36.2087326943557, '%')
(36.315228966986155, '%')
(36.42172523961661, '%')
(36.52822151224707, '%')
(36.63471778487753, '%')
(36.74121405750799, '%')
(36.84771033013845, '%')
(36.9542066027689, '%')
(37.06070287539936, '%')
(37.167199148029816, '%')
(37.273695420660275, '%')
(37.38019169329074, '%')
(37.48668796592119, '%')
(37.59318423855165, '%')
(37.69968051118211, '%')
(37.80617678381257, '%')
(37.912673056443026, '%')
(38.019169329073485, '%')
(38.12566560170394, '%')
(38.2321618743343

(68.79659211927583, '%')
(68.90308839190628, '%')
(69.00958466453673, '%')
(69.1160809371672, '%')
(69.22257720979765, '%')
(69.32907348242811, '%')
(69.43556975505857, '%')
(69.54206602768903, '%')
(69.6485623003195, '%')
(69.75505857294995, '%')
(69.86155484558041, '%')
(69.96805111821087, '%')
(70.07454739084132, '%')
(70.18104366347177, '%')
(70.28753993610223, '%')
(70.39403620873269, '%')
(70.50053248136315, '%')
(70.6070287539936, '%')
(70.71352502662407, '%')
(70.82002129925452, '%')
(70.92651757188499, '%')
(71.03301384451545, '%')
(71.1395101171459, '%')
(71.24600638977637, '%')
(71.35250266240682, '%')
(71.45899893503727, '%')
(71.56549520766772, '%')
(71.67199148029819, '%')
(71.77848775292864, '%')
(71.8849840255591, '%')
(71.99148029818956, '%')
(72.09797657082002, '%')
(72.20447284345049, '%')
(72.31096911608094, '%')
(72.4174653887114, '%')
(72.52396166134186, '%')
(72.63045793397231, '%')
(72.73695420660276, '%')
(72.84345047923323, '%')
(72.94994675186368, '%')
(73.05

![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**: Steven Duivenvoorden 

For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).

The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)